# 🧠 **Módulo 2: Fundamentos de Embeddings - A Linguagem dos Computadores**

> *Como transformar palavras em "coordenadas" que o computador entende*

---

## **Aula 2.1: Embeddings Básicos - Como Transformar Texto em Números**

---

### **Tá, mas o que são embeddings?**

Embeddings são como **tradutores universais** que transformam palavras em números que o computador entende. É tipo transformar "gato" em coordenadas como (0.2, -0.8, 0.5, ...).

**🖼️ Sugestão de imagem**: Palavras sendo transformadas em coordenadas no espaço

**Por que embeddings são importantes?**

Imagine que você quer encontrar roupas similares em uma loja:
- 👕 **Sem embeddings**: "Procure por camisetas azuis" (muito específico)
- 🧠 **Com embeddings**: "Encontre roupas com estilo similar" (inteligente)

### **Analogia do Dia a Dia**

Embeddings são como um **sistema de coordenadas geográficas**:
- 🌍 **São Paulo**: (-23.5505, -46.6333)
- 🌍 **Rio de Janeiro**: (-22.9068, -43.1729)
- �� **Gato**: (0.2, -0.8, 0.5, ...)
- �� **Cachorro**: (0.3, -0.7, 0.4, ...)

**Palavras similares ficam próximas no espaço!**

---

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou o notebook `00_setup_colab.ipynb` primeiro, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0
!pip install matplotlib>=3.5.0
!pip install scikit-learn>=1.3.0

print("✅ Dependências instaladas com sucesso!")
print("🚀 Ambiente configurado para Embeddings!")

In [ ]:
# 🧠 IMPORTAÇÕES PARA EMBEDDINGS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

# LangChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings

# Utilitários
import os
from dotenv import load_dotenv

print("✅ Bibliotecas importadas com sucesso!")
print("�� Pronto para explorar o mundo dos embeddings!")

### **Exemplo Prático - Embeddings Simples**

Vamos começar com um exemplo simples para entender o conceito:

In [ ]:
# 🧪 EXEMPLO PRÁTICO: EMBEDDINGS SIMPLES

# Simulando embeddings simples (para demonstração)
def criar_embeddings_simples(palavras):
    """Cria embeddings simples para demonstração"""
    
    # Dicionário de embeddings simulados
    embeddings_simples = {
        "gato": [0.8, 0.2, 0.1],
        "cachorro": [0.7, 0.3, 0.1],
        "peixe": [0.1, 0.9, 0.2],
        "pássaro": [0.2, 0.8, 0.3],
        "carro": [0.9, 0.1, 0.8],
        "moto": [0.8, 0.1, 0.9],
        "feliz": [0.1, 0.1, 0.9],
        "triste": [0.1, 0.1, 0.1],
        "grande": [0.5, 0.5, 0.5],
        "pequeno": [0.2, 0.2, 0.2]
    }
    
    return {palavra: embeddings_simples.get(palavra, [0, 0, 0]) for palavra in palavras}

# Testando embeddings simples
palavras_teste = ["gato", "cachorro", "peixe", "carro", "feliz"]
embeddings_simples = criar_embeddings_simples(palavras_teste)

print("�� EMBEDDINGS SIMPLES (demonstração):")
print("=" * 50)

for palavra, embedding in embeddings_simples.items():
    print(f"📝 '{palavra}' → {embedding}")

print(f"\n📊 Dimensão dos embeddings: {len(embeddings_simples['gato'])}")

### **Calculando Similaridade**

Agora vamos ver como calcular similaridade entre embeddings:

In [ ]:
# �� CALCULANDO SIMILARIDADE ENTRE EMBEDDINGS

def calcular_similaridade(embedding1, embedding2):
    """Calcula similaridade cosseno entre dois embeddings"""
    
    # Convertendo para arrays numpy
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    # Similaridade cosseno
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0
    
    return dot_product / (norm1 * norm2)

# Testando similaridades
print("�� SIMILARIDADE ENTRE PALAVRAS:")
print("=" * 50)

comparacoes = [
    ("gato", "cachorro"),
    ("gato", "peixe"),
    ("carro", "moto"),
    ("feliz", "triste"),
    ("gato", "carro")
]

for palavra1, palavra2 in comparacoes:
    sim = calcular_similaridade(
        embeddings_simples[palavra1], 
        embeddings_simples[palavra2]
    )
    
    print(f"�� '{palavra1}' vs '{palavra2}': {sim:.3f}")
    
    # Interpretação
    if sim > 0.8:
        print(f"   ✅ Muito similares!")
    elif sim > 0.5:
        print(f"   🤝 Similares")
    elif sim > 0.2:
        print(f"   🤔 Pouco similares")
    else:
        print(f"   ❌ Muito diferentes")
    print()

### **Visualizando Embeddings**

Vamos criar uma visualização para entender melhor como os embeddings se relacionam:

In [ ]:
# �� VISUALIZANDO EMBEDDINGS

def visualizar_embeddings(embeddings_dict):
    """Cria visualização 2D dos embeddings"""
    
    # Preparando dados
    palavras = list(embeddings_dict.keys())
    embeddings = list(embeddings_dict.values())
    
    # Reduzindo para 2D (se necessário)
    if len(embeddings[0]) > 2:
        tsne = TSNE(n_components=2, random_state=42)
        embeddings_2d = tsne.fit_transform(embeddings)
    else:
        embeddings_2d = embeddings
    
    # Criando gráfico
    plt.figure(figsize=(12, 8))
    
    # Plotando pontos
    x_coords = [coord[0] for coord in embeddings_2d]
    y_coords = [coord[1] for coord in embeddings_2d]
    
    plt.scatter(x_coords, y_coords, s=100, alpha=0.7)
    
    # Adicionando labels
    for i, palavra in enumerate(palavras):
        plt.annotate(palavra, (x_coords[i], y_coords[i]), 
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=12, fontweight='bold')
    
    plt.title('🧠 Visualização dos Embeddings', fontsize=16, fontweight='bold')
    plt.xlabel('Dimensão 1')
    plt.ylabel('Dimensão 2')
    plt.grid(True, alpha=0.3)
    
    # Adicionando legenda
    plt.figtext(0.02, 0.02, 
                '�� Palavras similares ficam próximas no espaço!', 
                fontsize=10, style='italic')
    
    plt.tight_layout()
    plt.show()

# Visualizando nossos embeddings simples
print("📈 Criando visualização dos embeddings...")
visualizar_embeddings(embeddings_simples)

print("✅ Visualização criada! Observe como palavras similares ficam próximas!")

## **Aula 2.2: Embeddings Avançados - Otimizando a Qualidade**

---

### **Tá, mas como funcionam embeddings reais?**

Embeddings reais são criados por **redes neurais** que aprendem a representar palavras baseado no contexto. É como um tradutor que entende nuances e significados.

**🖼️ Sugestão de imagem**: Rede neural processando texto

### **Setup Inicial - Preparando o Terreno**

Vamos configurar embeddings reais:

In [ ]:
# �� CONFIGURANDO EMBEDDINGS REAIS

def configurar_embeddings():
    """Configura modelo de embeddings real"""
    
    try:
        print("🚀 Carregando Sentence Transformers...")
        
        # Modelo pequeno e rápido para demonstração
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        
        print("✅ Embeddings configurados com sucesso!")
        return embeddings
        
    except Exception as e:
        print(f"⚠️ Erro ao carregar embeddings: {e}")
        print("💡 Vamos usar embeddings simulados para continuar")
        return None

# Configurando embeddings
embeddings_real = configurar_embeddings()

if embeddings_real:
    print(f"📊 Dimensão dos embeddings: {embeddings_real.client.get_sentence_embedding_dimension()}")
else:
    print("⚠️ Usando embeddings simulados para demonstração")

### **Exemplo Prático - Embeddings Reais**

Agora vamos testar embeddings reais:

In [ ]:
# 🧪 EXEMPLO PRÁTICO: EMBEDDINGS REAIS

def testar_embeddings_reais():
    """Testa embeddings reais com frases"""
    
    # Frases para teste
    frases_teste = [
        "O gato está dormindo no sofá",
        "O cachorro está brincando no jardim",
        "O peixe está nadando no aquário",
        "O carro está estacionado na garagem",
        "A pessoa está feliz com a notícia",
        "A pessoa está triste com a perda"
    ]
    
    if embeddings_real:
        print("🧪 TESTANDO EMBEDDINGS REAIS:")
        print("=" * 50)
        
        # Gerando embeddings
        embeddings_gerados = embeddings_real.embed_documents(frases_teste)
        
        print(f"✅ Gerados {len(embeddings_gerados)} embeddings")
        print(f"📊 Dimensão de cada embedding: {len(embeddings_gerados[0])}")
        
        # Calculando similaridades
        print("\n📊 SIMILARIDADES ENTRE FRASES:")
        
        comparacoes_frases = [
            (0, 1, "Gato vs Cachorro"),
            (0, 2, "Gato vs Peixe"),
            (3, 3, "Carro vs Carro (mesmo)"),
            (4, 5, "Feliz vs Triste")
        ]
        
        for i, j, descricao in comparacoes_frases:
            sim = cosine_similarity(
                [embeddings_gerados[i]], 
                [embeddings_gerados[j]]
            )[0][0]
            
            print(f"🔍 {descricao}: {sim:.3f}")
            
            if sim > 0.8:
                print(f"   ✅ Muito similares!")
            elif sim > 0.5:
                print(f"   🤝 Similares")
            elif sim > 0.2:
                print(f"   🤔 Pouco similares")
            else:
                print(f"   ❌ Muito diferentes")
            print()
        
        return embeddings_gerados, frases_teste
        
    else:
        print("⚠️ Embeddings reais não disponíveis")
        return None, None

# Testando embeddings reais
embeddings_reais, frases = testar_embeddings_reais()

### **Visualizando Embeddings Reais**

Vamos criar uma visualização dos embeddings reais:

In [ ]:
# �� VISUALIZANDO EMBEDDINGS REAIS

if embeddings_reais is not None:
    print("📈 Criando visualização dos embeddings reais...")
    
    # Reduzindo para 2D
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(embeddings_reais)-1))
    embeddings_2d = tsne.fit_transform(embeddings_reais)
    
    # Criando gráfico
    plt.figure(figsize=(14, 10))
    
    # Plotando pontos
    x_coords = embeddings_2d[:, 0]
    y_coords = embeddings_2d[:, 1]
    
    plt.scatter(x_coords, y_coords, s=150, alpha=0.7, c='skyblue', edgecolors='navy')
    
    # Adicionando labels
    for i, frase in enumerate(frases):
        # Abreviando frase para melhor visualização
        label = frase[:20] + "..." if len(frase) > 20 else frase
        plt.annotate(label, (x_coords[i], y_coords[i]), 
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    plt.title('🧠 Visualização dos Embeddings Reais', fontsize=16, fontweight='bold')
    plt.xlabel('Dimensão 1 (TSNE)')
    plt.ylabel('Dimensão 2 (TSNE)')
    plt.grid(True, alpha=0.3)
    
    # Adicionando legenda
    plt.figtext(0.02, 0.02, 
                '💡 Frases com significado similar ficam próximas!', 
                fontsize=10, style='italic')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualização criada! Observe como frases similares se agrupam!")

### **Teste Rápido**

Vamos fazer um teste rápido para encontrar frases similares:

In [ ]:
# 🧪 TESTE RÁPIDO: ENCONTRANDO FRASES SIMILARES

def encontrar_frases_similares(frase_consulta, frases, embeddings, top_k=3):
    """Encontra as frases mais similares a uma consulta"""
    
    if embeddings_real:
        # Gerando embedding da consulta
        embedding_consulta = embeddings_real.embed_query(frase_consulta)
        
        # Calculando similaridades
        similaridades = []
        for i, embedding in enumerate(embeddings):
            sim = cosine_similarity([embedding_consulta], [embedding])[0][0]
            similaridades.append((i, sim))
        
        # Ordenando por similaridade
        similaridades.sort(key=lambda x: x[1], reverse=True)
        
        print(f"�� Consulta: '{frase_consulta}'")
        print(f"�� Top {top_k} frases mais similares:")
        print("=" * 50)
        
        for rank, (idx, sim) in enumerate(similaridades[:top_k], 1):
            print(f"{rank}. {frases[idx]}")
            print(f"   Similaridade: {sim:.3f}")
            print()
        
    else:
        print("⚠️ Embeddings reais não disponíveis para este teste")

# Testando busca por similaridade
if embeddings_reais is not None:
    print("�� TESTE DE BUSCA POR SIMILARIDADE:")
    print("=" * 50)
    
    consultas_teste = [
        "Um animal doméstico",
        "Um veículo motorizado",
        "Uma emoção positiva"
    ]
    
    for consulta in consultas_teste:
        encontrar_frases_similares(consulta, frases, embeddings_reais)
        print("-" * 30)

### **Desafio do Módulo**

Agora é sua vez! Crie seus próprios embeddings:

In [ ]:
# 🎯 DESAFIO DO MÓDULO

print("�� DESAFIO: Crie seus próprios embeddings!")
print("=" * 40)
print("1. Adicione suas próprias frases na lista abaixo")
print("2. Execute o código para gerar embeddings")
print("3. Teste similaridades entre suas frases")
print("=" * 40)

# Suas frases personalizadas
minhas_frases = [
    # Adicione aqui frases sobre um tema que você gosta
    # Exemplo: filmes, música, esportes, tecnologia, etc.
    "",
    "",
    "",
    "",
    ""
]

# Descomente e execute quando tiver suas frases:
# if embeddings_real and any(minhas_frases):
#     print("\n�� GERANDO EMBEDDINGS PARA SUAS FRASES:")
#     meus_embeddings = embeddings_real.embed_documents(minhas_frases)
#     print(f"✅ Gerados {len(meus_embeddings)} embeddings")
#     
#     # Teste de similaridade
#     consulta_pessoal = "Sua consulta aqui"
#     encontrar_frases_similares(consulta_pessoal, minhas_frases, meus_embeddings)

## **�� Módulo 2 Concluído!**

### **✅ O que aprendemos:**

1. **🧠 O que são embeddings**: Transformação de texto em números
2. **📊 Similaridade**: Como calcular proximidade entre textos
3. **📈 Visualização**: Como entender relações entre embeddings
4. **🚀 Embeddings reais**: Usando modelos profissionais
5. **🔍 Busca por similaridade**: Encontrando textos relacionados

### **�� Próximos Passos:**

No próximo módulo vamos aprender sobre **Vector Stores** - onde guardar todos esses embeddings de forma eficiente!

### **💡 Dicas Importantes:**

- **Embeddings capturam significado** - não apenas palavras
- **Similaridade cosseno** é a métrica mais usada
- **Dimensões maiores** = mais precisão (mas mais lento)
- **Modelos pré-treinados** são melhores que treinar do zero

---

**🎯 Desafio do Módulo**: Crie embeddings para suas próprias frases!

**💡 Dica do Pedro**: Embeddings são como GPS para palavras - cada palavra tem suas coordenadas no espaço!

**🚀 Próximo módulo**: Vector Stores - A memória inteligente!